In [92]:
import sys
sys.path.append('../')

from hvo_sequence.hvo_seq import HVO_Sequence

import os
import numpy as np
import pickle

In [93]:
source_path = "../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs"
print(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"))
train_file = open(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_sequence_data.obj"),'rb')
train_set = pickle.load(train_file)
dataset_size = len(train_set)
# ix =  int(np.random.random_sample()*dataset_size)
example = train_set[52] # 4-4 120 bpm 2 bars
example.to_html_plot(show_figure=True)

../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs/GrooveMIDI_processed_train/hvo_data.obj


Figure(id='1288', ...)

In [138]:
# https://github.com/mcartwright/dafx2018_adt/blob/master/large_vocab_adt_dafx2018/features.py

import librosa
import scipy.signal
import numpy as np
import resampy
import soundfile as psf

# IN ORDER TO CALC HOP SIZE
FRAME_INTERVAL = 0.01  # s

def read_audio(filepath, sr=None, mono=True, peak_norm=False):
    """
    Read audio
    Parameters
    ----------
    filepath
    sr
    mono
    Returns
    -------
    y, sr
    """
    try:
        y, _sr = psf.read(filepath)
        y = y.T
    except RuntimeError:
        y, _sr = librosa.load(filepath, mono=False, sr=None)

    if sr is not None and sr != _sr:
        y = resampy.resample(y, _sr, sr, filter='kaiser_fast')
    else:
        sr = _sr

    if mono:
        y = librosa.to_mono(y)

    if peak_norm:
        y /= np.max(np.abs(y))

    return y, sr


def cq_matrix(bins_per_octave, num_bins, f_min, fft_len, sr):
    """
    Compute center frequencies of the log-spaced filterbank
    Parameters
    ----------
    bins_per_octave : int
    num_bins : int
    f_min : float
    fft_len : int
    sr : float
    Returns
    -------
    c_mat
    """
    # note range goes from -1 to bpo*num_oct for boundary issues
    f_cq = f_min * 2 ** ((np.arange(-1, num_bins+1)) / bins_per_octave)         # center frequencies
    # centers in bins
    kc = np.round(f_cq * (fft_len / sr)).astype(int)
    c_mat = np.zeros([num_bins, int(np.round(fft_len / 2))])
    for k in range(1, kc.shape[0]-1):
        l1 = kc[k]-kc[k-1]
        w1 = scipy.signal.triang((l1 * 2) + 1)
        l2 = kc[k+1]-kc[k]                             
        w2 = scipy.signal.triang((l2 * 2) + 1)
        wk = np.hstack([w1[0:l1], w2[l2:]])  # concatenate two halves. l1 and l2 are different because of the log-spacing
        c_mat[k-1, kc[k-1]:(kc[k+1]+1)] = wk / np.sum(wk)  # normalized to unit sum;
    return c_mat, f_cq        # matrix with triangular filterbank


def onset_detection_fn(x, f_win_size, f_hop_size, f_bins_per_octave, f_octaves, f_fmin, sr, mean_filter_size):
    """
    Filter bank for onset pattern calculation
    """
    # calculate frequency constant-q transform
    f_win = scipy.signal.hann(f_win_size)
    x_spec = librosa.stft(x,
                          n_fft=f_win_size,
                          hop_length=f_hop_size,
                          win_length=f_win_size,
                          window=f_win)
    x_spec = np.abs(x_spec) / (2 * np.sum(f_win))

    f_cq_mat,f_cq = cq_matrix(f_bins_per_octave, f_octaves * f_bins_per_octave, f_fmin, f_win_size, sr)
    x_cq_spec = np.dot(f_cq_mat, x_spec[:-1, :])

    # subtract moving mean
    # DIFFERENCE BETWEEN THE CURRENT FRAME AND THE MEAN OF THE PREVIOUS mean_filter_size FRAMES 
    b = np.concatenate([[1], np.ones(mean_filter_size, dtype=float) / -mean_filter_size])    
    od_fun = scipy.signal.lfilter(b, 1, x_cq_spec, axis=1)                                      

    # half-wave rectify
    od_fun = np.maximum(0, od_fun)

    # post-process OPs
    od_fun = np.log10(1 + 1000*od_fun)                  ## log scaling
    
    
    logf_stft = librosa.power_to_db(x_cq_spec).astype('float32')
    od_fun = np.abs(od_fun).astype('float32')

    # reorder axis
    logf_stft= np.moveaxis(logf_stft, 1, 0)
    od_fun = np.moveaxis(od_fun, 1, 0)
    
    # clip od_fun
    od_fun = np.clip(od_fun/2.25, 0, 1)    # 2.25 ?????????
    
    
    return od_fun, logf_stft, f_cq

In [139]:
def reduce_frequency_bands_in_spectrogram(freq_out, freq_in, S):
    """
    @param freq_out:        band center frequencies in output spectrogram
    @param freq_in:         band center frequencies in input spectrogram
    @param S:               spectrogram to reduce
    @returns S_out:         spectrogram reduced in frequency
    """
    
    if len(freq_out) >= len(freq_in):
        warnings.warn("Number of bands in reduced spectrogram should be smaller than initial number of bands in spectrogram")
    
    n_timeframes = S.shape[0]
    n_bands = len(freq_out)
    
    # find index of closest input frequency
    freq_out_idx = np.array([],dtype=int)

    for f in freq_out:
        freq_out_idx = np.append(freq_out_idx, np.abs(freq_in - f).argmin())
        

    # band limits (not center)
    freq_out_band_idx = np.array([0], dtype=int)

    for i in range(len(freq_out_idx)-1):
        li = np.ceil((freq_out_idx[i+1] - freq_out_idx[i]) / 2) + freq_out_idx[i]      # find left border of band 
        freq_out_band_idx = np.append(freq_out_band_idx, [li])
    
    freq_out_band_idx = np.append(freq_out_band_idx,len(freq_in))    # add last frequency in input spectrogram
    freq_out_band_idx = np.array(freq_out_band_idx,dtype=int)     # convert to int

    
    # init empty spectrogram
    S_out = np.zeros([n_timeframes, n_bands])
    
    # reduce spectrogram
    for i in range(len(freq_out_band_idx)-1):
        li = freq_out_band_idx[i]+1        # band left index
        if i == 0: li = 0           
        ri = freq_out_band_idx[i+1]        # band right index
        S_out[:, i] = np.max(S[:,li:ri],axis=1)   # pooling

    return S_out

In [140]:
def get_grid_timestamps(n_bars = 2, time_signature_numerator = 4, time_signature_denominator = 4, 
                        beat_division_factors = [4],qpm = 120):
    
    """
    @param n_bars                               Number of bars
    @param time_signature_numerator             Time signature numerator
    @param time_signature_denominator           Time signature denominator
    @param beat_division_factors                Array of beat division factors
    @param qpm                                  Quarter per minute
    @return grid                                Array of timestamps of grid lines
    
    """
    
    hvo_seq = HVO_Sequence()
    hvo_seq.add_time_signature(time_step=0, numerator=time_signature_numerator, denominator=time_signature_denominator, beat_division_factors=beat_division_factors)
    hvo_seq.add_tempo(time_step=0, qpm = qpm)

    #total number of steps is total_number_of_bars*time_signature_numerator*beat_division_factor
    n_timesteps = 0
    for factor in beat_division_factors: 
        n_timesteps += n_bars*time_signature_numerator*factor

    # Create a zero hvo
    hits = np.zeros([n_timesteps, 1])
    vels = np.zeros([n_timesteps, 1])
    offs = np.zeros([n_timesteps, 1])

    # Add hvo score to hvo_seq instance
    hvo_seq.hvo = np.concatenate((hits, vels, offs), axis=1)

    # get grid lines
    #hvo_seq.grid_lines_with_types
    grid = hvo_seq.grid_lines
    #grid_len = hvo_seq.total_len
    
    return grid

In [141]:
def get_onset_detect(onset_strength):
    """
    
    """
    n_timeframes = onset_strength.shape[0]
    n_bands = onset_strength.shape[1]
    
    onset_detect = np.zeros([n_timeframes, n_bands])

    for band in range(n_bands):
        time_frame_idx = librosa.onset.onset_detect(onset_envelope=onset_strength.T[band,:])
        onset_detect[time_frame_idx,band] = 1

    return onset_detect
    

In [142]:
import warnings
from hvo_sequence.io_helpers import get_grid_position_and_utiming_in_hvo

def map_onsets_to_grid(grid, onset_strength, onset_detect):
    """
    Maps matrices of onset strength and onset detection into a grid with a lower temporal resolution.
    @param grid:                 Array with timestamps
    @param onset_strength:       Matrix of onset strength values (n_timeframes x n_bands)
    @param onset_detect:         Matrix of onset detection (1,0) (n_timeframes x n_bands)
    @return onsets_grid:         Onsets with respect to lines in grid (len_grid x n_bands)
    @return intensity_grid:      Strength values for each detected onset (len_grid x n_bands)
    """
    
    if onset_strength.shape != onset_detect.shape:
        warnings.warn(f"onset_strength shape and onset_detect shape must be equal. Instead, got {onset_strength.shape} and {onset_detect.shape}")
    
    n_bands = onset_strength.shape[1]
    n_timeframes = onset_detect.shape[0]
    n_timesteps = len(grid)


    # init intensity and onsets grid
    strength_grid = np.zeros([n_timesteps + 1,n_bands])
    onsets_grid = np.zeros([n_timesteps + 1,n_bands])
    
    # time array
    time = librosa.frames_to_time(np.arange(n_timeframes), sr=sr, 
                              hop_length=int(round(FRAME_INTERVAL * sr)), n_fft=1024)

    # map onsets and strength into grid
    for band in range(n_bands):
        for timeframe_idx in range(n_timeframes):       
            if onset_detect[timeframe_idx,band]:         # if there is an onset detected, get grid index and utiming
                grid_idx, utiming = get_grid_position_and_utiming_in_hvo(time[timeframe_idx],grid)
                strength_grid[grid_idx, band] = onset_strength[timeframe_idx,band]
                onsets_grid[grid_idx, band] = utiming
                
    
    return onsets_grid, strength_grid

In [143]:
sr=44100
audio_file_path = "./misc/temp.wav"

# IN ORDER TO CALC HOP SIZE
FRAME_INTERVAL = 0.01  # s

x, sr = read_audio(audio_file_path, mono=True, sr=sr)
x /= np.max(np.abs(x))

f_win_size = 1024
f_hop_size = int(round(FRAME_INTERVAL * sr))
f_bins_per_octave =16
f_octaves = 9
f_fmin = 40
mean_filter_size = 22



od_fun, logf_stft, f_cq = onset_detection_fn(x,
                                             f_win_size,
                                             f_hop_size,
                                             f_bins_per_octave,
                                             f_octaves,
                                             f_fmin,
                                             sr,
                                             mean_filter_size)

mb_onset_strength = od_fun

c_freq = [55, 90, 138, 175, 350, 6000,8500,12500]                      # Perfe center frequencies
mb_onset_strength = reduce_frequency_bands_in_spectrogram(c_freq, f_cq, mb_onset_strength)
mb_onset_detect = get_onset_detect(mb_onset_strength)
grid = get_grid_timestamps(n_bars = 2, time_signature_numerator = 4, 
                                      time_signature_denominator = 4, beat_division_factors = [4],qpm = 120)

onsets_grid, strength_grid = map_onsets_to_grid(grid, mb_onset_strength, mb_onset_detect, n_fft=n_fft, hop_length = hop_length)

In [144]:
print(np.round(onsets_grid,2))

[[ 0.33  0.33  0.25  0.25  0.41  0.33  0.17  0.17]
 [ 0.37  0.37 -0.11  0.    0.05  0.37  0.21  0.37]
 [ 0.    0.    0.    0.    0.    0.    0.33  0.41]
 [ 0.    0.    0.    0.   -0.43  0.   -0.27  0.29]
 [ 0.49  0.49  0.09  0.33  0.01  0.25  0.41  0.25]
 [ 0.29  0.29 -0.19  0.    0.   -0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.01  0.17]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.49  0.25  0.01  0.01  0.01  0.01  0.01  0.01]
 [ 0.05  0.13 -0.35  0.    0.    0.05  0.    0.  ]
 [ 0.    0.    0.   -0.07  0.    0.01  0.25  0.17]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.41  0.41  0.49  0.49  0.17  0.17  0.49  0.33]
 [ 0.21  0.29 -0.19  0.    0.    0.   -0.35 -0.43]
 [ 0.    0.    0.    0.    0.    0.25  0.25  0.41]
 [ 0.    0.21  0.21  0.21  0.21  0.    0.    0.  ]
 [ 0.41  0.41  0.17  0.17  0.17  0.17  0.17  0.17]
 [ 0.21  0.29 -0.27  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.09  0.09  0.25]
 [ 0.    0.    0.    0.    0.  

In [145]:
print(np.round(strength_grid, 2))

[[0.98 0.96 0.53 0.49 0.45 0.6  0.88 0.86]
 [0.4  0.25 0.34 0.   0.27 0.7  0.82 0.22]
 [0.   0.   0.   0.   0.   0.   0.42 0.78]
 [0.   0.   0.   0.   0.18 0.   0.47 0.19]
 [0.92 0.82 0.95 0.8  0.51 0.91 0.93 0.19]
 [0.37 0.36 0.27 0.   0.   0.57 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.64 0.65]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.65 0.76 0.8  0.63 0.35 0.42 0.16 0.3 ]
 [0.24 0.19 0.35 0.   0.   0.12 0.   0.  ]
 [0.   0.   0.   0.11 0.   0.28 0.38 0.93]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.87 0.78 0.64 0.68 0.87 0.66 0.78 0.28]
 [0.35 0.09 0.31 0.   0.   0.   0.45 0.2 ]
 [0.   0.   0.   0.   0.   0.12 0.68 0.66]
 [0.   0.12 0.18 0.22 0.21 0.   0.   0.  ]
 [0.84 0.77 0.8  0.63 0.36 0.41 0.17 0.31]
 [0.28 0.22 0.26 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.14 0.52 0.43]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.87 0.8  0.71 0.75 0.85 1.   0.96 0.42]
 [0.37 0.16 0.33 0.   0.   0.71 0.59 0.  ]
 [0.   0.   0.   0.   0.   0.1  0.75 0.78]
 [0.   0.  